# Glacier mapping with deep learning

In [ ]:
!pip install rasterio

In [ ]:
import tensorflow as tf
import h5py
import pandas
import numpy as np
import rasterio
import pickle
import subprocess
import geopandas
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

# !UPDATE root_dir!
root_dir = os.path.join("gdrive", "My Drive", "Advanced Image Analysis", "Glacier mapping exercise")
import sys
sys.path.append(root_dir)

In [ ]:
import dataloaders
import models.mapping
import losses
import metrics
import utils.deeplearning
import utils.misc

In [ ]:
# configuration variables
batch_size = 4
max_epochs = 3 # !ADJUST FOR YOUR OWN EXPERIMENT!
learning_rate = 5e-4

patch_size = 384
n_features = 10
n_classes = 2
input_shape = (patch_size, patch_size, n_features)

# !UPDATE data_folder!
data_folder = os.path.join("gdrive", "My Drive", "Finse", "2023")

### Setting up dataloaders

In [ ]:
# open subsets
train = h5py.File(os.path.join(data_folder, "train.hdf5"))
val = h5py.File(os.path.join(data_folder, "val.hdf5"))

In [ ]:
# define training dataloader with random sampler and on-fly data augmentation
train_dataloader = dataloaders.DataLoader(
    dataloaders.RandomSampler(train, patch_size),
    [
        dataloaders.Augmentation([
            dataloaders.transformations.random_vertical_flip(),
            dataloaders.transformations.random_horizontal_flip(),
            dataloaders.transformations.random_rotation(),
            dataloaders.transformations.crop_and_scale(patch_size=patch_size),
        ]),
    ],
    batch_size,
)

steps_per_epoch = len(train_dataloader)

In [ ]:
# define validation dataloader with consecutive (deterministic) sampler
val_dataloader = dataloaders.DataLoader(
    dataloaders.ConsecutiveSampler(val, patch_size),
    [],
    batch_size,
)

In [ ]:
batch_x, batch_y = train_dataloader[42]
_, axs = plt.subplots(nrows=6, ncols=batch_size, figsize=(10, batch_size * 3.5))

for col in range(batch_size):
    axs[0][col].imshow(utils.misc.norm_to_vis(batch_x["features"][col][:, :, [5, 3, 2]]), aspect="auto")
    axs[1][col].imshow(utils.misc.norm_to_vis(batch_x["features"][col][:, :, 6]), aspect="auto")
    axs[2][col].imshow(utils.misc.norm_to_vis(batch_x["features"][col][:, :, 7]), aspect="auto")
    axs[3][col].imshow(utils.misc.norm_to_vis(batch_x["features"][col][:, :, 8]), aspect="auto")
    axs[4][col].imshow(utils.misc.norm_to_vis(batch_x["features"][col][:, :, 9]), aspect="auto")
    axs[5][col].imshow(batch_y[col][:, :, 1], aspect="auto", vmin=0, vmax=1)

plt.tight_layout()
plt.show()

In [ ]:
batch_x, batch_y = val_dataloader[42]
_, axs = plt.subplots(nrows=6, ncols=batch_size, figsize=(10, batch_size * 3.5))

for col in range(batch_size):
    axs[0][col].imshow(utils.misc.norm_to_vis(batch_x["features"][col][:, :, [5, 3, 2]]), aspect="auto")
    axs[1][col].imshow(utils.misc.norm_to_vis(batch_x["features"][col][:, :, 6]), aspect="auto")
    axs[2][col].imshow(utils.misc.norm_to_vis(batch_x["features"][col][:, :, 7]), aspect="auto")
    axs[3][col].imshow(utils.misc.norm_to_vis(batch_x["features"][col][:, :, 8]), aspect="auto")
    axs[4][col].imshow(utils.misc.norm_to_vis(batch_x["features"][col][:, :, 9]), aspect="auto")
    axs[5][col].imshow(batch_y[col][:, :, 1], aspect="auto", vmin=0, vmax=1)

plt.tight_layout()
plt.show()

In [ ]:
# reset validation daaloader for further use
val_dataloader.sampler.reset()

### Defining model and training

In [ ]:
model = models.mapping.DeepLabMini(input_shape, n_classes, name="DeepLabMini")
model.summary()

In [ ]:
# 'compile' model by providing optimisation algorithm, loss and metrics to track during training
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=losses.FocalLoss(),
    metrics=[
        tf.keras.metrics.CategoricalAccuracy(),
        tf.keras.metrics.Precision(class_id=1),
        tf.keras.metrics.Recall(class_id=1),
        metrics.IoU(class_id=1),
    ],
)

In [ ]:
# define callbacks for saving logs, model weights, early stopping and learning rate schedule
training_callbacks = [
    utils.deeplearning.LRWarmup(
        warmup_steps=steps_per_epoch,
        target=learning_rate,
        verbose=1,
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_iou",
        mode="max",
        factor=0.1,
        patience=10,
        verbose=1,
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_iou",
        mode="max",
        patience=31,
        verbose=1,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        f"{model.name}_weights.h5",
        monitor=f"val_iou",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
    ),
    tf.keras.callbacks.CSVLogger(f"{model.name}_log.csv"),
]

In [ ]:
# finally, train model
model.fit(
    train_dataloader,
    epochs=max_epochs,
    validation_data=val_dataloader,
    callbacks=training_callbacks,
    verbose=1
)

In [ ]:
# close subset
train.close()
val.close()

### Evaluating model

In [ ]:
# load model with best weights
model = models.mapping.ResUNet(input_shape, n_classes, name="ResUNetMini_pretrained")
model.load_weights(os.path.join(root_dir, "weights", f"{model.name}_weights.h5"))

In [ ]:
# train/val iou curves
history = pandas.read_csv(os.path.join(root_dir, "logs", f"{model.name}_log.csv"))

plt.plot(history["epoch"] + 1, history["iou"], label="training")
plt.plot(history["epoch"] + 1, history["val_iou"], label="validation")
plt.xlabel("# epochs")
plt.ylabel("Intersection over union")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# open test subset
test = h5py.File(os.path.join(data_folder, "test.hdf5"), "r")

In [ ]:
# classify test samples
results = {}

for tile in tqdm(test.keys()):
    group = test[tile]

    features = np.array(group["features"])
    outlines = np.array(group["outlines"])

    prediction = utils.deeplearning.apply(
        features, model, patch_size=patch_size, batch_size=batch_size, n_outputs=n_classes
    )
    prediction = np.argmax(prediction, axis=-1)

    pad_height, pad_width = group.attrs["pad_height"], group.attrs["pad_width"]
    height, width = group.attrs["height"], group.attrs["width"]

    features = features[pad_height:pad_height + height, pad_width:pad_width + width, :]
    outlines = outlines[pad_height:pad_height + height, pad_width:pad_width + width, 1]
    prediction = prediction[pad_height:pad_height + height, pad_width:pad_width + width]

    results[tile] = {
        "thumbnail": features.reshape((height, width, n_features))[:, :, [5, 3, 2]],
        "outlines": outlines,
        "prediction": prediction
    }

In [ ]:
# close test subset
test.close()

In [ ]:
# calculate metrics
tp, fp, fn = 0, 0, 0

for tile, result in tqdm(results.items()):
    groundtruth = result["outlines"]
    prediction = result["prediction"]

    tp_mask = (prediction == 1) & (groundtruth == 1)
    fp_mask = (prediction == 1) & (groundtruth == 0)
    fn_mask = (prediction == 0) & (groundtruth == 1)

    tp += np.sum(tp_mask)
    fp += np.sum(fp_mask)
    fn += np.sum(fn_mask)

precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
iou = tp / (tp + fp + fn)

print(f"precision = {precision}")
print(f"recall = {recall}")
print(f"f1-score = {f1}")
print(f"iou = {iou}")

In [ ]:
# visualise some tiles
tiles_to_visualise = [
    "ALP-23-20",
    "HMA-110-78",
    "HMA-134-89",
    "NZ1-17-28",
    "SA-152-37",
]

nrows, ncols = 2, len(tiles_to_visualise)
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(4 * ncols, 4.5 * nrows))

for col, tile in enumerate(tiles_to_visualise):
    axes[0][col].set_title(tile)

    thumbnail = utils.misc.norm_to_vis(results[tile]["thumbnail"])
    axes[0][col].imshow(thumbnail, aspect="auto")

    groundtruth = results[tile]["outlines"]
    prediction = results[tile]["prediction"]

    output_colors = ["black", "white", "red", "green"]
    output_cmap = mpl.colors.ListedColormap(output_colors, 4)
    output = np.zeros(prediction.shape)
    output[(prediction == 1) & (groundtruth == 1)] = 1
    output[(prediction == 0) & (groundtruth == 1)] = 2
    output[(prediction == 1) & (groundtruth == 0)] = 3
    axes[1][col].imshow(
        output / 3, vmin=0, vmax=1, aspect="auto", cmap=output_cmap,
        interpolation="none"
    )

for row in range(nrows):
    for col in range(ncols):
        axes[row][col].axis("off")

labels = ["True negative", "True positive", "False negative", "False positive"]
handles = [mpl.patches.Patch(facecolor=_, edgecolor="black") for _ in output_colors]

fig.legend(handles, labels, ncol=4, loc="lower center", bbox_to_anchor=(0.5, -0.022))

plt.tight_layout()
plt.show()

### Applying trained model

In [ ]:
# load data stack for HMA1
with rasterio.open(os.path.join(data_folder, "raster", "HMA1", "stack.tif")) as src:
    stack = src.read()
    stack = np.moveaxis(stack, 0, -1)
    meta = src.meta.copy()

In [ ]:
# normalize features
with open(os.path.join(data_folder, "mins_maxs.pickle"), "rb") as min_max_file:
    mins, maxs = pickle.load(min_max_file)

stack = (stack - mins) / (maxs - mins)
stack[np.isnan(stack)] = 0
stack[np.isinf(stack)] = 0

In [ ]:
# pad stack to fit patch size
height, width, _ = stack.shape
new_height = (height // patch_size + 1) * patch_size
new_width = (width // patch_size + 1) * patch_size
pad_height = (new_height - height) // 2
pad_width = (new_width - width) // 2

stack_pad = np.zeros((new_height, new_width, n_features))
stack_pad[pad_height:pad_height + height, pad_width:pad_width + width, :] = stack

In [ ]:
# apply model
prediction = utils.deeplearning.apply(
    stack_pad, model, patch_size=patch_size, batch_size=batch_size, n_outputs=n_classes
)
prediction = np.argmax(prediction, axis=-1)
prediction = prediction[pad_height:pad_height + height, pad_width:pad_width + width]

In [ ]:
# save raster output
meta.update({
    "dtype": np.uint8,
    "count": 1,
    "nodata": 0
})

raster_output_path = "hma1_prediction.tif"
with rasterio.open(raster_output_path, "w", **meta) as dst:
    dst.write(prediction, 1)

In [ ]:
vector_output_path = "hma1_prediction.shp"
subprocess.run(f"""
    bash -c '
    python {root_dir}/utils/geo/polygonize.py {raster_output_path} {vector_output_path}
    '
""", shell=True)

In [ ]:
vector_prediction = geopandas.read_file(vector_output_path)
vector_prediction.plot()